In [1]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable
import time
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

import numpy as np
import aeon
import torch
from torch import Tensor
import torch.nn as nn
import torch.functional as F
import pandas as pd
from aeon.datasets.tsc_datasets import univariate_equal_length, multivariate_equal_length
univariate_equal_length = sorted(list(univariate_equal_length))
multivariate_equal_length = sorted(list(multivariate_equal_length))
from aeon.datasets import load_classification
from sklearn.linear_model import RidgeCV, RidgeClassifierCV
from sklearn.metrics import accuracy_score
from tqdm import tqdm

from utils.utils import print_name, print_shape
from preprocessing.stream_transforms import normalize_mean_std_traindata, normalize_streams, augment_time, add_basepoint_zero
from random_sig_fourier import SigTensorisedRandProj
from signature import SigTransform, LogSigTransform
from features.base import TimeseriesFeatureExtractor, TabularTimeseriesFeatures, RandomGuesser
from randomized_sig import RandomizedSignature
from rocket_wrappers import RocketWrapper, MiniRocketWrapper, MultiRocketWrapper

np.set_printoptions(precision=3, threshold=5) # Print options

/home/nikita/Code/zephyrox/.conda/lib/python3.11/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


In [2]:
#############################################
#######          Dataset Code         #######
#############################################

def get_aeon_dataset(
        dataset_name:str, 
        extract_path = "/home/nikita/hdd/Data/TSC/",
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        ):
    """Loads a dataset from the UCR/UEA archive using 
    the aeon library.

    Args:
        dataset_name (str): Name of the dataset

    Returns:
        Tuple: 4-tuple of the form (X_train, y_train, X_test, y_test)
    """
    X_train, y_train = load_classification(dataset_name, split="train", extract_path=extract_path)
    X_test, y_test = load_classification(dataset_name, split="test", extract_path=extract_path)
    X_train = torch.from_numpy(X_train.transpose(0,2,1)).to(device)
    X_test = torch.from_numpy(X_test.transpose(0,2,1)).to(device)
    return X_train, y_train, X_test, y_test

In [3]:
# def test_get_data(idx: int = 17):
#     name = univariate_equal_length[idx]
#     X_train, y_train, X_test, y_test = get_aeon_dataset(name, device="cpu")
#     print("Dataset:", name)
#     print("idx:", idx)
#     print("X_train", X_train.shape)
#     print("X_test", X_test.shape)

# for i in range(20):
#     test_get_data(i)
#     print("\n")

In [4]:
##################################
####  Linear Model (Ridge)  ######
##################################

def train_and_test_linear(
        train_X, train_y, test_X, test_y,
        feat_extractor: TimeseriesFeatureExtractor,
        apply_augmentation:bool=True,
        normalize_features:bool=True,
        clf=RidgeClassifierCV(alphas=np.logspace(-3, 3, 20))
    ):
    # augment data
    print(train_X.shape)
    if apply_augmentation:
        train_X, test_X = normalize_mean_std_traindata(train_X, test_X)
        train_X = add_basepoint_zero(train_X)
        train_X = augment_time(train_X)
        test_X = add_basepoint_zero(test_X)
        test_X = augment_time(test_X)

    # fit transformer
    t0 = time.time()
    feat_extractor.fit(train_X)
    feat_train_X = feat_extractor.transform(train_X).cpu().numpy()
    feat_test_X = feat_extractor.transform(test_X).cpu().numpy()
    print("feat_train_X", feat_train_X.shape)
    if normalize_features:
        feat_train_X, feat_test_X = normalize_mean_std_traindata(feat_train_X, feat_test_X)


    # feed into linear classifier
    t1 = time.time()
    clf.fit(feat_train_X, train_y)
    t2 = time.time()

    # predict
    pred = clf.predict(feat_test_X)
    test_acc = accuracy_score(test_y, pred)
    train_acc = accuracy_score(train_y, clf.predict(feat_train_X))
    alpha = clf.alpha_ if hasattr(clf, 'alpha_') else None
    return train_acc, test_acc, alpha, t1-t0, t2-t1

In [5]:
def run_allModels_singleDataset(X_train, y_train, X_test, y_test):
    max_batch = 32
    trunc_level = 4
    n_features = 1344

    models = [
        ["Random Guesser", RandomGuesser()],
        ["Tabular", TabularTimeseriesFeatures()],
        # ["Sig", SigTransform(trunc_level, max_batch)],
        # ["Log Sig", LogSigTransform(trunc_level, max_batch)],
        ["Randomized Signature", RandomizedSignature(
            n_features,
            activation = "tanh",
            max_batch=10,
            )],
        ["TRP", SigTensorisedRandProj(
            trunc_level,
            n_features,
            only_last=True,
            method="linear",
            max_batch=max_batch,
            )],
        ["TRP rbf", SigTensorisedRandProj(
            trunc_level,
            n_features,
            only_last=True,
            method="RBF",
            sigma_rbf=1.0,
            max_batch=max_batch,
            )],
        ["concat TRP", SigTensorisedRandProj(
            trunc_level,
            n_features // (trunc_level-1),
            only_last=False,
            method="linear",
            max_batch=max_batch,
            )],
        ["concat TRP rbf", SigTensorisedRandProj(
            trunc_level,
            n_features // (trunc_level-1),
            only_last=False,
            method="RBF",
            sigma_rbf=1.0,
            max_batch=max_batch,
            )],
        ["Rocket", RocketWrapper(
            n_features
            )],
        ["MiniRocket", MiniRocketWrapper(
            n_features
            )],
        ["MultiRocket", MultiRocketWrapper(
            n_features
            )],
        ]

    # Run experiments
    model_names = [name for (name, _) in models]
    results_ridge = []
    for name, model in models:
        print("name", name)
        result = train_and_test_linear(
            X_train, y_train, X_test, y_test, model
            )
        results_ridge.append(result)
        print()
    
    return model_names, results_ridge

In [6]:
def run_dataset(dataset_name:str):
    X_train, y_train, X_test, y_test = get_aeon_dataset(dataset_name)
    X_train, X_test = normalize_streams(X_train, X_test, max_T=1000)
    model_names, results_ridge = run_allModels_singleDataset(X_train, y_train, X_test, y_test)
    return model_names, results_ridge

model_names, results_ridge = run_dataset(univariate_equal_length[9])

name Random Guesser
torch.Size([20, 24, 1])
feat_train_X (20, 2)

name Tabular
torch.Size([20, 24, 1])
feat_train_X (20, 50)

name Randomized Signature
torch.Size([20, 24, 1])
feat_train_X (20, 1344)

name TRP
torch.Size([20, 24, 1])
feat_train_X (20, 1344)

name TRP rbf
torch.Size([20, 24, 1])
feat_train_X (20, 1344)

name concat TRP
torch.Size([20, 24, 1])
feat_train_X (20, 1344)

name concat TRP rbf
torch.Size([20, 24, 1])
feat_train_X (20, 1344)

name Rocket
torch.Size([20, 24, 1])
feat_train_X (20, 1344)

name MiniRocket
torch.Size([20, 24, 1])
feat_train_X (20, 1344)

name MultiRocket
torch.Size([20, 24, 1])
feat_train_X (20, 1344)



In [7]:
from rocket_wrappers import RocketWrapper, MiniRocketWrapper, MultiRocketWrapper

for n_features in [100, 500, 1000, 2000, 1344, 4000, 6000, 8000, 10000]:
    rocket = RocketWrapper(n_features)
    mini_rocket = MiniRocketWrapper(n_features)
    multi_rocket = MultiRocketWrapper(n_features)
    
    print("n_features:", n_features)
    
    # Generate random input
    input_shape = (2, 30, 1)  # Example shape, modify as needed
    X = torch.randn(input_shape)
    
    # Fit to random input
    print(rocket.fit_transform(X).shape)
    print(mini_rocket.fit_transform(X).shape)
    print(multi_rocket.fit_transform(X).shape)
    print()

n_features: 100
torch.Size([2, 100])
torch.Size([2, 84])
torch.Size([2, 672])

n_features: 500
torch.Size([2, 500])
torch.Size([2, 420])
torch.Size([2, 672])

n_features: 1000
torch.Size([2, 1000])
torch.Size([2, 924])
torch.Size([2, 672])

n_features: 2000
torch.Size([2, 2000])
torch.Size([2, 1932])
torch.Size([2, 1344])

n_features: 1344
torch.Size([2, 1344])
torch.Size([2, 1344])
torch.Size([2, 1344])

n_features: 4000
torch.Size([2, 4000])
torch.Size([2, 3948])
torch.Size([2, 3360])

n_features: 6000
torch.Size([2, 6000])
torch.Size([2, 5964])
torch.Size([2, 5376])

n_features: 8000
torch.Size([2, 8000])
torch.Size([2, 7980])
torch.Size([2, 7392])

n_features: 10000
torch.Size([2, 10000])
torch.Size([2, 9996])
torch.Size([2, 9408])



In [8]:
def run_allModels_allData(datasets: List[str]):
    #run experiments
    experiments = {}
    failed = {}
    for dataset_name in tqdm(datasets):
        t0 = time.time()
        try:
            print(dataset_name)
            X_train, y_train, X_test, y_test = get_aeon_dataset(dataset_name)
            X_train, X_test = normalize_streams(X_train, X_test, max_T=1000)
            N_train = X_train.shape[0]
            N_test = X_test.shape[0]
            T = X_train.shape[1]
            D = X_train.shape[2]
            if N_train<=2000 and D<=20:
                results = run_allModels_singleDataset(
                    X_train, y_train, X_test, y_test
                    )
                experiments[dataset_name] = results
        except Exception as e:
            print(f"Error: {e}")
            failed[dataset_name] = e
        print("Elapsed time", time.time()-t0)
    
    #parse results
    # Define the attributes and methods
    attributes = ["ACC_train", "ACC_test", "alpha", "time_transform", "time_fit"]
    
    # Extract model_names from d_res
    model_names = next(iter(experiments.values()))[0]

    # Create and save DataFrames for each attribute and method
    for attribute in attributes:
        df = pd.DataFrame(columns=model_names)
        for dataset_name, (model_names, results_ridge) in experiments.items():
            values = [res[attributes.index(attribute)] for res in results_ridge]
            df.loc[dataset_name] = values

        # Save the DataFrame
        print(df)
        df.to_pickle(f"TSC_{attribute}_results.pkl")

    return experiments, failed

In [9]:
run_allModels_allData(univariate_equal_length[:10])

  0%|          | 0/10 [00:00<?, ?it/s]

ACSF1
name Random Guesser
torch.Size([100, 730, 1])
feat_train_X (100, 2)

name Tabular
torch.Size([100, 730, 1])
feat_train_X (100, 1462)

name Randomized Signature
torch.Size([100, 730, 1])
feat_train_X (100, 1344)

name TRP
torch.Size([100, 730, 1])
feat_train_X (100, 1344)

name TRP rbf
torch.Size([100, 730, 1])
feat_train_X (100, 1344)

name concat TRP
torch.Size([100, 730, 1])
feat_train_X (100, 1344)

name concat TRP rbf
torch.Size([100, 730, 1])
feat_train_X (100, 1344)

name Rocket
torch.Size([100, 730, 1])
feat_train_X (100, 1344)

name MiniRocket
torch.Size([100, 730, 1])
feat_train_X (100, 1344)

name MultiRocket
torch.Size([100, 730, 1])


 10%|█         | 1/10 [00:39<05:54, 39.37s/it]

feat_train_X (100, 1344)

Elapsed time 39.37058162689209
Adiac
name Random Guesser
torch.Size([390, 176, 1])
feat_train_X (390, 2)

name Tabular
torch.Size([390, 176, 1])
feat_train_X (390, 354)

name Randomized Signature
torch.Size([390, 176, 1])
feat_train_X (390, 1344)

name TRP
torch.Size([390, 176, 1])
feat_train_X (390, 1344)

name TRP rbf
torch.Size([390, 176, 1])
feat_train_X (390, 1344)

name concat TRP
torch.Size([390, 176, 1])
feat_train_X (390, 1344)

name concat TRP rbf
torch.Size([390, 176, 1])
feat_train_X (390, 1344)

name Rocket
torch.Size([390, 176, 1])
feat_train_X (390, 1344)

name MiniRocket
torch.Size([390, 176, 1])
feat_train_X (390, 1344)

name MultiRocket
torch.Size([390, 176, 1])


 20%|██        | 2/10 [01:16<05:06, 38.32s/it]

...

 90%|█████████ | 9/10 [02:52<00:18, 18.33s/it]

feat_train_X (60, 1344)

Elapsed time 19.759113311767578
Chinatown
name Random Guesser
torch.Size([20, 24, 1])
feat_train_X (20, 2)

name Tabular
torch.Size([20, 24, 1])
feat_train_X (20, 50)

name Randomized Signature
torch.Size([20, 24, 1])
feat_train_X (20, 1344)

name TRP
torch.Size([20, 24, 1])
feat_train_X (20, 1344)

name TRP rbf
torch.Size([20, 24, 1])
feat_train_X (20, 1344)

name concat TRP
torch.Size([20, 24, 1])
feat_train_X (20, 1344)

name concat TRP rbf
torch.Size([20, 24, 1])
feat_train_X (20, 1344)

name Rocket
torch.Size([20, 24, 1])
feat_train_X (20, 1344)

name MiniRocket
torch.Size([20, 24, 1])
feat_train_X (20, 1344)

name MultiRocket
torch.Size([20, 24, 1])


100%|██████████| 10/10 [02:55<00:00, 17.52s/it]

feat_train_X (20, 1344)

Elapsed time 2.8462674617767334
             Random Guesser   Tabular  Randomized Signature       TRP  \
ACSF1              0.160000  0.550000              0.550000  0.320000   
Adiac              0.056410  0.584615              0.384615  0.179487   
ArrowHead          0.500000  0.944444              0.694444  0.611111   
BME                0.466667  1.000000              0.666667  0.966667   
Beef               0.333333  1.000000              0.666667  0.600000   
BeetleFly          0.750000  0.900000              0.850000  0.950000   
BirdChicken        0.600000  0.750000              0.900000  0.950000   
CBF                0.400000  1.000000              1.000000  0.900000   
Car                0.283333  0.966667              0.900000  0.816667   
Chinatown          0.600000  1.000000              1.000000  1.000000   

              TRP rbf  concat TRP  concat TRP rbf    Rocket  MiniRocket  \
ACSF1        0.490000    0.340000        0.530000  0.850000    0

({'ACSF1': (['Random Guesser',
    'Tabular',
    'Randomized Signature',
    'TRP',
    'TRP rbf',
    'concat TRP',
    'concat TRP rbf',
    'Rocket',
    'MiniRocket',
    'MultiRocket'],
   [(0.16, 0.1, 1000.0, 0.0006437301635742188, 0.002818584442138672),
    (0.55, 0.3, 1000.0, 0.0019047260284423828, 0.025457143783569336),
    (0.55, 0.51, 483.2930238571752, 8.899884700775146, 0.02023768424987793),
    (0.32, 0.31, 0.1623776739188721, 0.4139738082885742, 0.02761054039001465),
    (0.49, 0.45, 483.2930238571752, 22.98890447616577, 0.00937032699584961),
    (0.34, 0.32, 483.2930238571752, 0.14661192893981934, 0.012362480163574219),
    (0.53, 0.5, 483.2930238571752, 3.020693778991699, 0.007422208786010742),
    (0.85, 0.71, 233.57214690901213, 2.661346435546875, 0.04980754852294922),
    (0.91, 0.67, 233.57214690901213, 0.6189532279968262, 0.04858660697937012),
    (0.87,
     0.61,
     483.2930238571752,
     0.2647526264190674,
     0.039434194564819336)]),
  'Adiac': (['Random

In [10]:
# Define the attributes and methods
attributes = ["ACC_train", "ACC_test", "time_transform", "time_fit", "alpha"]
#data_dir = "https://github.com/nikitazozoulenko/zephyrox/raw/main/Data/TSER/"
data_dir = ""
# Load and store the DataFrames for each attribute and method
dfs = {}
for attribute in attributes:
    filename = f"TSC_{attribute}_results.pkl"
    print(data_dir+filename)
    df = pd.read_pickle(data_dir + filename)
    dfs[attribute] = df

TSC_ACC_train_results.pkl
TSC_ACC_test_results.pkl
TSC_time_transform_results.pkl
TSC_time_fit_results.pkl
TSC_alpha_results.pkl


In [11]:
dfs["ACC_test"]

,Random Guesser,Tabular,Randomized Signature,TRP,TRP rbf,concat TRP,concat TRP rbf,Rocket,MiniRocket,MultiRocket
ACSF1,0.100000,0.300000,0.510000,0.310000,0.450000,0.320000,0.500000,0.710000,0.670000,0.610000
Adiac,0.033248,0.414322,0.281330,0.150895,0.291560,0.166240,0.337596,0.705882,0.721228,0.713555
ArrowHead,0.291429,0.737143,0.302857,0.377143,0.571429,0.491429,0.508571,0.680000,0.800000,0.817143
BME,0.360000,0.940000,0.453333,0.733333,0.660000,0.733333,0.580000,0.973333,0.946667,0.946667
Beef,0.266667,0.866667,0.400000,0.400000,0.533333,0.433333,0.466667,0.833333,0.800000,0.566667
BeetleFly,0.500000,0.800000,0.800000,0.750000,0.800000,0.800000,0.800000,0.850000,0.950000,0.850000
BirdChicken,0.500000,0.550000,0.750000,0.700000,0.750000,0.750000,0.750000,1.000000,1.000000,1.000000
CBF,0.331111,0.834444,0.910000,0.847778,0.958889,0.856667,0.945556,0.995556,0.991111,0.951111
Car,0.216667,0.800000,0.566667,0.466667,0.650000,0.533333,0.633333,0.816667,0.850000,0.816667
Chinatown,0.448980,0.982507,0.941691,0.944606,0.842566,0.915452,0.897959,0.959184,0.953353,0.965015
